In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
from scipy.stats import rankdata
import torch
import dgl
import time
from tqdm import tqdm
import random
import gc

/home/hst/anaconda3/envs/gnn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GIN(torch.nn.Module):
    def __init__(self, in_dim, layer_num=3):
        super(GIN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.activations = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        for i in range(layer_num):
            self.convs.append(
                dgl.nn.GINConv(torch.nn.Sequential(
                    torch.nn.Linear(in_dim, in_dim, bias=False),
                    torch.nn.BatchNorm1d(in_dim),
                    torch.nn.LeakyReLU(),
                    torch.nn.Linear(in_dim, in_dim, bias=False),
                ), learn_eps=False))
            self.activations.append(torch.nn.LeakyReLU())
            self.batch_norms.append(torch.nn.BatchNorm1d(in_dim))

        self.layer_num = layer_num
        self.out_dim = in_dim * (layer_num+1)
        
    def forward(self, g, h):
        hs = [h]
        for conv, batch_norm, act in zip(self.convs, self.batch_norms, self.activations):
            h = conv(g, h)
            h = batch_norm(h)
            h = act(h)
            hs.append(h)
        return torch.cat(hs, dim=-1)

class GNNModel(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, dropout_rate=0.5):
        super(GNNModel, self).__init__()
        self.comp = torch.nn.Sequential(
            torch.nn.Linear(in_dim, hidden_dim),
            torch.nn.LeakyReLU()
        )
        self.gcn = GIN(hidden_dim*2)
        self.Act_head = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.gcn.out_dim*2, self.gcn.out_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.gcn.out_dim, 1),
        )
        self.Sel_head = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.gcn.out_dim*2, self.gcn.out_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.gcn.out_dim, 1),
        )
    
    def forward(self, g, wildtype_seq, mutation_seq, mutation_pos):
        wildtype_h = self.comp(wildtype_seq)
        mutation_h = self.comp(mutation_seq)
        cat_h = torch.cat([wildtype_h, mutation_h], dim=-1)
        
        cat_h = self.gcn(g, cat_h)
        
        with g.local_scope():
            g.ndata['h'] = cat_h
            cat_hg = dgl.readout_nodes(g, 'h', op='sum')
         
        mutation_pos = mutation_pos.float().unsqueeze(1)
        with g.local_scope():
            g.ndata['h'] = cat_h*mutation_pos
            cat_hp = dgl.readout_nodes(g, 'h', op='sum')
        
        h_all = torch.cat([cat_hg, cat_hp], dim=-1)
        
        pred_Act = self.Act_head(h_all)
        pred_Sel = self.Sel_head(h_all)
        
        return pred_Act, pred_Sel

In [3]:
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, dir_path, indexes=None, add_self_loop=False):
        super(GraphDataset, self).__init__()
        self.dir_path = dir_path
        self.graphs, label_dict = dgl.load_graphs(self.dir_path+'/dgl_graph.bin')
        self.df = pd.read_csv(self.dir_path+'/overview_df.csv', index_col=0)
        self.add_self_loop = add_self_loop
        if indexes is None:
            self.indexes = self.df.index
        else:
            self.indexes = indexes

    def __getitem__(self, i):
        idx = self.indexes[i]
        
        row = self.df.loc[idx]
        
        graph_index = row.graph_index
        graph = self.graphs[graph_index].clone()
        if self.add_self_loop:
            graph = dgl.add_self_loop(graph)
        
        wildtype_feature = np.load(self.dir_path+'/'+row.wildtype_feature_path)
        wildtype_seq = wildtype_feature['wildtype_seq']
        
        mutation_feature = np.load(self.dir_path+'/'+row.mutation_feature_path)
        mutation_seq = mutation_feature['mutation_seq']
        mutation_pos = mutation_feature['mutation_pos']
        
        graph.ndata['wildtype_seq'] = torch.from_numpy(wildtype_seq)
        graph.ndata['mutation_seq'] = torch.from_numpy(mutation_seq)
        graph.ndata['mutation_pos'] = torch.from_numpy(mutation_pos)
        
        Act = row.get('Normalized Activity', np.nan)
        Act_valid = True
        
        if np.isnan(Act):
            Act_valid = False
         
        Sel = row.get('Normalized Selectivity', np.nan)
        Sel_valid = True
        if np.isnan(Sel):
            Sel_valid = False

        
        return graph, Act, Act_valid, Sel, Sel_valid, idx

    def __len__(self):
        return len(self.indexes)

In [4]:
def one_fold(train_indexes, val_indexes, cv_num=0):
    print(f'CV{cv_num}==================================')
    
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    dgl.seed(seed)
    
    os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 这里的0是GPU设备的编号
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    hidden_dim = 256
    epochs = 70 #70
    batch_size = 16
    lr = 3e-4
    patience = 50
    weight_decay = 0.0
    
    train_dataset = GraphDataset('train_dataset', train_indexes)
    train_dataloader = dgl.dataloading.GraphDataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=os.cpu_count())
    val_dataset = GraphDataset('train_dataset', val_indexes)
    val_dataloader = dgl.dataloading.GraphDataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=os.cpu_count())
    graph, Act, Act_valid, Sel, Sel_valid, original_index = next(iter(train_dataloader))
    feature_dim = graph.ndata['wildtype_seq'].shape[-1]
    model = GNNModel(feature_dim, hidden_dim).to(device)
    criterion = torch.nn.L1Loss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    st = time.time()
    min_val_loss = 1e5
    min_not_update_count = 0
    for epoch in range(epochs):
        all_Act_loss = 0
        all_Sel_loss = 0
        Act_count = 0
        Sel_count = 0
        model.train()
        for graph, Act, Act_valid, Sel, Sel_valid, original_index in tqdm(train_dataloader, leave=True):
            graph = graph.to(device)
            #print(graph.device)

            Act = Act.float().to(device).unsqueeze(1)
            Sel = Sel.float().to(device).unsqueeze(1)

            wildtype_seq, mutation_seq, mutation_pos = graph.ndata['wildtype_seq'], graph.ndata['mutation_seq'], graph.ndata['mutation_pos']
            pred_Act, pred_Sel = model(graph, wildtype_seq, mutation_seq, mutation_pos)
            Act_loss = criterion(pred_Act[Act_valid], Act[Act_valid])
            Sel_loss = criterion(pred_Sel[Sel_valid], Sel[Sel_valid])
            loss = Act_loss + Sel_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            all_Act_loss += Act_loss.item()
            all_Sel_loss += Sel_loss.item()
            Act_count += Act.float().sum()
            Sel_count += Sel.float().sum()
        train_Act_loss = all_Act_loss / Act_count
        train_Sel_loss = all_Sel_loss / Sel_count
        train_all_loss = train_Act_loss + train_Sel_loss

        all_Act_loss = 0
        all_Sel_loss = 0
        Act_count = 0
        Sel_count = 0
        model.eval()
        for graph, Act, Act_valid, Sel, Sel_valid, original_index in tqdm(val_dataloader, leave=True):
            graph = graph.to(device)

            Act = Act.float().to(device).unsqueeze(1)
            Sel = Sel.float().to(device).unsqueeze(1)

            
            wildtype_seq, mutation_seq, mutation_pos = graph.ndata['wildtype_seq'], graph.ndata['mutation_seq'], graph.ndata['mutation_pos']
            with torch.no_grad():
                pred_Act, pred_Sel = model(graph, wildtype_seq, mutation_seq, mutation_pos)
                Act_loss = criterion(pred_Act[Act_valid], Act[Act_valid])
                Sel_loss = criterion(pred_Sel[Sel_valid], Sel[Sel_valid])
            all_Act_loss += Act_loss.item()
            all_Sel_loss += Sel_loss.item()
            Act_count += Act.float().sum()
            Sel_count += Sel.float().sum()
        val_Act_loss = all_Act_loss / Act_count
        val_Sel_loss = all_Sel_loss / Sel_count
        val_all_loss = val_Act_loss + val_Sel_loss
        spent_time = time.time() - st
        print(f'epoch: {epoch+1}  train_loss: {train_all_loss} train_Act_loss: {train_Act_loss} train_Sel_loss: {train_Sel_loss}')
        print(f'time: {np.around(spent_time/60, decimals=2)}min val_loss: {val_all_loss} val_Act_loss: {val_Act_loss} val_Sel_loss: {val_Sel_loss}')
        if val_Act_loss <= min_val_loss:
            min_val_loss = val_Act_loss
            min_not_update_count = 0
            torch.save(model.state_dict(), f'best_model{cv_num}.pth')
        else:
            min_not_update_count += 1
        if min_not_update_count > patience:
            print('Early stopping!')
            break
        if spent_time > 6000:
            print('timeover')
            break
      
    test_dataset = GraphDataset('test_dataset')
    test_dataloader = dgl.dataloading.GraphDataLoader(test_dataset, batch_size=16, shuffle=False, drop_last=False, num_workers=os.cpu_count())
    sub_df = test_dataset.df.copy()
    sub_df['act'] = None
    model.load_state_dict(torch.load(f'best_model{cv_num}.pth'))

    model.eval()

    for graph, Act, Act_valid, Sel, Sel_valid, original_index in test_dataloader:
        graph = graph.to(device)
        wildtype_seq, mutation_seq, mutation_pos = graph.ndata['wildtype_seq'], graph.ndata['mutation_seq'], graph.ndata['mutation_pos']
        with torch.no_grad():
            pred_Act, pred_Sel = model(graph, wildtype_seq, mutation_seq, mutation_pos)
        pred_Act = pred_Act.cpu().numpy()
        pred_Sel = pred_Sel.cpu().numpy()


        original_index = original_index.numpy()
        sub_df.loc[original_index, 'act'] = pred_Act
        sub_df.loc[original_index, 'sel'] = pred_Sel

    print(f'CV{cv_num} finish! min_val_loss={min_val_loss}')
    
    sub_df['seq_id'] = sub_df.index
    #print(sub_df)
    return sub_df[['seq_id', 'act', 'sel']]

In [5]:
df = pd.read_csv('train_dataset/overview_df.csv', index_col=0)
df

,mutant_seq,Normalized Activity,Normalized Selectivity,mutation_feature_path,wildtype_feature_path,graph_index
0,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.000,1.000,mutation_feature_0.npz,wildtype_feature.npz,0
1,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,3.228,1.837,mutation_feature_1.npz,wildtype_feature.npz,1
2,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,2.170,2.445,mutation_feature_2.npz,wildtype_feature.npz,2
3,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.759,1.061,mutation_feature_3.npz,wildtype_feature.npz,3
4,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.531,1.032,mutation_feature_4.npz,wildtype_feature.npz,4
...,...,...,...,...,...,...
1589,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.044,1.879,mutation_feature_1589.npz,wildtype_feature.npz,1589
1590,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.042,2.051,mutation_feature_1590.npz,wildtype_feature.npz,1590
1591,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.045,1.939,mutation_feature_1591.npz,wildtype_feature.npz,1591
1592,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.038,1.887,mutation_feature_1592.npz,wildtype_feature.npz,1592


In [6]:
indexes = df.index
hasAct = df.loc[indexes, 'Normalized Activity'].isna().values
group = df.loc[indexes, 'graph_index'].values
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
sub_df = None
column_names_act = []
column_names_sel = []

for cv_num, (train, val) in enumerate(skf.split(indexes, hasAct, group)):
    train_indexes, val_indexes = indexes[train], indexes[val]
    
    column_name_act = f'act{cv_num}'
    column_name_sel = f'sel{cv_num}'
    
    cv_sub_df = one_fold(train_indexes, val_indexes, cv_num).rename(columns={'act': column_name_act, 'sel' : column_name_sel})
    
    column_names_act.append(column_name_act)
    column_names_sel.append(column_name_sel)
    
    if sub_df is None:
        sub_df = cv_sub_df
    else:
        sub_df = pd.merge(sub_df, cv_sub_df, on='seq_id')
        
    gc.collect()

CV0==================================


100%|███████████████████████████████████████████████████████████████████████| 79/79 [00:06<00:00, 12.71it/s]

100%|███████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 16.07it/s]


epoch: 1  train_loss: 75.98918151855469 train_Act_loss: 62.106990814208984 train_Sel_loss: 13.882187843322754

time: 0.12min val_loss: 3.011837959289551 val_Act_loss: 1.8001524209976196 val_Sel_loss: 1.2116854190826416


100%|███████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 20.48it/s]

100%|███████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 14.20it/s]


epoch: 2  train_loss: 2.65714693069458 train_Act_loss: 1.9529305696487427 train_Sel_loss: 0.7042163610458374

time: 0.21min val_loss: 1.1200177669525146 val_Act_loss: 0.7914936542510986 val_Sel_loss: 0.3285241723060608


...

epoch: 68  train_loss: 0.8876445889472961 train_Act_loss: 0.6140225529670715 train_Sel_loss: 0.2736220359802246

time: 5.36min val_loss: 0.8109374046325684 val_Act_loss: 0.5056771039962769 val_Sel_loss: 0.3052603006362915


100%|███████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 22.83it/s]

100%|███████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 15.18it/s]


epoch: 69  train_loss: 0.8981900215148926 train_Act_loss: 0.6134136319160461 train_Sel_loss: 0.2847764194011688

time: 5.44min val_loss: 5.425581455230713 val_Act_loss: 4.769874572753906 val_Sel_loss: 0.6557070016860962


100%|███████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 23.50it/s]

100%|███████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 15.46it/s]


epoch: 70  train_loss: 0.9687181711196899 train_Act_loss: 0.6568909883499146 train_Sel_loss: 0.311827152967453

time: 5.51min val_loss: 0.8173443675041199 val_Act_loss: 0.48493218421936035 val_Sel_loss: 0.3324121832847595

CV4 finish! min_val_loss=0.47018593549728394


In [7]:
sub_df

,seq_id,act0,sel0,act1,sel1,act2,sel2,act3,sel3,act4,sel4
0,0,0.626878,2.253832,0.610621,1.984168,0.830186,1.398671,0.710957,2.876400,0.67864,3.523947
1,1,0.615556,2.311088,0.601749,1.913226,0.803834,1.417565,0.709136,2.273335,0.667143,3.728196
2,2,0.657278,1.729700,0.581657,1.588208,0.761458,1.155476,0.664408,0.294419,0.580639,3.052859
3,3,0.597393,2.054880,0.585855,1.709753,0.623689,1.526616,0.575537,1.789287,0.50731,3.465257
4,4,0.583646,2.196716,0.56153,1.892435,0.584607,1.632855,0.585799,2.312948,0.506028,3.703316
...,...,...,...,...,...,...,...,...,...,...,...
920,920,0.143694,2.087167,0.15752,2.156046,0.161745,1.689106,0.104634,2.476568,0.068654,2.023842
921,921,0.124661,2.129577,0.161919,2.494146,0.135059,2.111785,0.108601,3.049519,0.082781,2.431201
922,922,0.064792,1.740550,0.067286,1.860313,0.106566,1.519276,0.040689,1.855434,0.030709,1.795904
923,923,0.080178,1.815779,0.095253,2.160872,0.129061,1.650182,0.053248,2.258700,0.034638,2.080630


In [8]:
sub_df.to_csv('submission/cv_submission_gnn.csv', index=False)

In [9]:
sub_df = pd.read_csv('submission/cv_submission_gnn.csv')
    #print(rankdata(sub_df[column_name_sel]))
    
sub_df['act'] = sub_df[column_names_act].mean(axis='columns')
sub_df['sel'] = sub_df[column_names_sel].mean(axis='columns')

sub_df = sub_df[['seq_id', 'act', 'sel']]
sub_df['seq_id'] = sub_df['seq_id'].apply(lambda x: x + 1594)
sub_df = sub_df.rename(columns={'seq_id': 'SequenceID', 'act': 'Activity', 'sel': 'Selectivity'})
sub_df

,SequenceID,Activity,Selectivity
0,1594,0.691456,2.407404
1,1595,0.679484,2.328682
2,1596,0.649088,1.564132
3,1597,0.577957,2.109159
4,1598,0.564322,2.347654
...,...,...,...
920,2514,0.127249,2.086546
921,2515,0.122604,2.443246
922,2516,0.062008,1.754296
923,2517,0.078476,1.993233


In [11]:
sub_df.to_csv('submission/submission_gnn.csv', index=False)